In [1]:
from utils import *

In [67]:
X,y = load_tweets('../data/clean_train_full.txt', True)

In [68]:
labels = ['__label__1' if l == 1 else '__label__-1' for l in y]

In [69]:
with open('../data/clean_train_full_fasttext.txt', 'w') as f:
    for l,x in zip(labels, X):
        f.write(l + ' ' + x + '\n')

* * *

In [2]:
all_tweets = load_tweets('../data/clean/train_full_fasttext.txt')

In [3]:
import numpy as np

def split_data(X, train_ratio, seed=None, verbose=False):
    if seed is not None: np.random.seed(seed)
    N = len(X)
    limit = int(N * train_ratio)
    indices = np.random.choice(range(N),N)
    train_indices = indices[:limit]
    test_indices = indices[limit:]
    X_train = [X[i] for i in train_indices]
    X_val = [X[i] for i in test_indices]
    if verbose:
        pos, neg = 0, 0
        for x in X_train:
            if '__label__1' in x: pos += 1
            else: neg += 1
        print('TRAIN %.2f%% Pos  %.2f%% Neg  (%g ratio)' % (pos/len(X_train)*100, neg/len(X_train)*100, pos/neg))
        pos, neg = 0, 0
        for x in X_val:
            if '__label__1' in x: pos += 1
            else: neg += 1
        print('VAL   %.2f%% Pos  %.2f%% Neg (%g ratio)' % (pos/len(X_val)*100, neg/len(X_val)*100, pos/neg))
    return X_train, X_val

In [4]:
X_train, X_val = split_data(all_tweets, 0.8, seed=0)

In [5]:
TRAIN_PATH = '../data/ft_train.txt'
save_tweets(TRAIN_PATH, X_train)
VAL_PATH = '../data/ft_val.txt'
save_tweets(VAL_PATH, X_val)

In [6]:
import fasttext
MODEL_PATH = '../models/ft.bin' # CHANGE THIS PATH
best = fasttext.load_model(MODEL_PATH)

In [7]:
best.test(VAL_PATH)

(454097, 0.8568367551426237, 0.8568367551426237)

In [8]:
def predict_all(model, test_data):
    labels, probas = [], []
    for x in test_data:
        l, p = model.predict(x)
        labels.append(int(l[0][9:]))
        probas.append(p[0])
    return labels, probas

In [191]:
generate_submission('../out/submission_ft_200.csv', labels)

In [190]:
best.test(TRAIN_PATH)

(1816385, 0.8567820148261519, 0.8567820148261519)

In [113]:
params = {
    'lr' : 0.1,
    'epoch': 5,
    'loss' : 'softmax',
    'wordNgrams': 3,
    'dim' : 200,
    'minCount' : 5
}

print(params)
for i in range(5):
    model = fasttext.train_supervised(TRAIN_PATH, **params)
    print(model.test(VAL_PATH))

params['minCount'] = 0

print(params)
for i in range(5):
    model = fasttext.train_supervised(TRAIN_PATH, **params)
    print(model.test(VAL_PATH))

params['loss'] = 'ns'
params['minCount'] = 5

print(params)
for i in range(5):
    model = fasttext.train_supervised(TRAIN_PATH, **params)
    print(model.test(VAL_PATH))

{'lr': 0.1, 'epoch': 5, 'loss': 'softmax', 'wordNgrams': 3, 'dim': 200, 'minCount': 5}
(454097, 0.8372131945377309, 0.8372131945377309)
(454097, 0.8143414292541021, 0.8143414292541021)
(454097, 0.8169443973424181, 0.8169443973424181)
(454097, 0.8324058516132016, 0.8324058516132016)
(454097, 0.8256980336800287, 0.8256980336800287)
{'lr': 0.1, 'epoch': 5, 'loss': 'softmax', 'wordNgrams': 3, 'dim': 200, 'minCount': 0}
(454097, 0.8369621468540862, 0.8369621468540862)
(454097, 0.8178318729258286, 0.8178318729258286)
(454097, 0.7235987024798667, 0.7235987024798667)
(454097, 0.8331171533835282, 0.8331171533835282)
(454097, 0.8265040288748879, 0.8265040288748879)
{'lr': 0.1, 'epoch': 5, 'loss': 'ns', 'wordNgrams': 3, 'dim': 200, 'minCount': 5}
(454097, 0.8120357544753654, 0.8120357544753654)
(454097, 0.6906696146418057, 0.6906696146418057)
(454097, 0.7327619429328976, 0.7327619429328976)
(454097, 0.8007275978480368, 0.8007275978480368)
(454097, 0.8077305069181254, 0.8077305069181254)


In [223]:
params = {
    'lr' : 0.05,
    'epoch': 5,
    'loss' : 'softmax',
    'wordNgrams': 3,
    'dim' : 200,
    'minCount' : 5
}

TRAIN_PATH = '../data/ft_train.txt'
VAL_PATH = '../data/ft_val.txt'

print(params)
for i in range(2):
    X_train, X_val = split_data(all_tweets, 0.8, seed=i)
    save_tweets(TRAIN_PATH, X_train)
    save_tweets(VAL_PATH, X_val)
    model = fasttext.train_supervised(TRAIN_PATH, **params)
    print(model.test(VAL_PATH))

{'lr': 0.05, 'epoch': 5, 'loss': 'softmax', 'wordNgrams': 3, 'dim': 200, 'minCount': 5}
(454097, 0.908658282261279, 0.908658282261279)
(454097, 0.9091537711105777, 0.9091537711105777)


In [224]:
model.save_model('../models/ft_lr005')

In [10]:
import fasttext
TRAIN_PATH = '../data/ft_train.txt'
VAL_PATH = '../data/ft_val.txt'

params = {
    'lr' : 0.05,
    'epoch': 5,
    'loss' : 'softmax',
    'wordNgrams': 3,
    'dim' : 200,
    'minCount' : 5,
    'verbose' : 2
}

In [9]:
test_data = load_tweets('../data/clean/test.txt')

In [11]:
nb_iter = 1
final_probas = np.zeros((len(test_data),))
pos_count = np.zeros((len(test_data),))
neg_count = np.zeros((len(test_data),))
for i in range(nb_iter):
    print('\rTraining %d / %d' % (i+1, nb_iter), end='')
    
    # only train on part of the data for each iteration (say 80 %) ...
    X_train, X_val = split_data(all_tweets, 0.8, seed=i)
    save_tweets(TRAIN_PATH, X_train)
    save_tweets(VAL_PATH, X_val)
    model = fasttext.train_supervised(TRAIN_PATH, **params)
    
    # ... or train on full dataset
    #model = fasttext.train_supervised('../data/clean/train_full_fasttext.txt', **params)
    
    # we can also check the accuracy on validation data (slows thigs down)
    # print('\r             Val acc: %g' % (model.test(VAL_PATH)[1]), end='')
    
    ls, ps = predict_all(model, test_data)
    ls = np.array(ls)
    pos_count += 1 * (ls > 0)
    neg_count += 1 * (ls < 0)
    final_probas += (ls * np.array(ps))
final_probas /= nb_iter
# final_labels = [1 if x >= 0 else -1 for x in final_probas]
# generate_submission('../out/submission_ft_'+str(nb_iter)+'_iter.csv', final_labels)

Training 1 / 1

In [17]:
model.test(VAL_PATH)

(454097, 0.9095721839166522, 0.9095721839166522)

In [22]:
model.get_

array([[ -0.90423965,   4.384958  ,   1.5792971 ,   2.8815448 ,
         -5.2630014 ,   4.6238346 ,   4.801919  ,   7.2931824 ,
         -1.8636836 ,  -6.210276  ,   6.431934  ,   0.2434191 ,
          4.1955423 ,  -1.2406583 ,   6.889752  ,   2.1614995 ,
         -3.30318   ,  -5.1521883 ,   3.9263895 ,  -2.5178695 ,
         -0.4202886 ,  -0.460544  ,   3.725308  ,   3.62996   ,
          0.48703283,   2.4030771 ,  -4.8485336 ,   9.03467   ,
          0.83473927,  -6.803135  ,   1.6415502 , -11.539529  ,
          2.5269089 ,   1.0341111 ,   2.4801059 ,   4.235219  ,
          1.4422336 ,  -6.7213435 ,  -2.2742248 ,   0.36974233,
          2.6136966 ,  -3.0076737 ,  -2.823734  ,  -0.34518144,
          1.5038188 ,  -2.4816272 ,  -7.598887  ,  -1.1529282 ,
         -4.3092976 ,  -4.1150093 ,   4.842186  ,  -1.8290958 ,
         -1.8663241 ,   2.3002288 ,   3.3506458 ,  -1.799013  ,
         -0.47564074,   7.8201127 ,   2.724961  ,   3.4218032 ,
         -0.52700245,  -6.5371943 ,  -4.

In [11]:
np.save('../out/final_probas_7_iter', final_probas)

In [20]:
majority = pos_count > neg_count
majority = 2 * majority - 1

array([-1, -1, -1, ..., -1,  1, -1])

In [22]:
final_labels = [1 if x >= 0 else -1 for x in final_probas]

In [23]:
sum(final_labels != majority)

16

In [25]:
generate_submission('../out/submission_ft_7_iter_majority.csv', majority)

* * *

In [1]:
import sent2vec
import numpy
import pickle
model = sent2vec.Sent2vecModel()
model.load_model('../models/embed-model-full.bin') # The model can be sent2vec or cbow-c+w-ngrams
uni_embs, vocab = model.get_unigram_embeddings() # Return the full unigram embedding matrix

#with open('full-vocab.pkl', 'wb') as f:
#    pickle.dump(vocab, f)

#numpy.save('full-vocab-matrix', uni_embs)

In [2]:
from utils import *

In [3]:
X = load_tweets('../data/clean_train_no_label.txt')

In [4]:
len(vocab)

42566

In [5]:
uni_embs.shape

(42566, 700)

In [6]:
word2vec = dict()
for i, w in enumerate(vocab):
    word2vec[w] = uni_embs[i]

In [21]:
word2index = {w:i for i,w in enumerate(vocab)}

In [18]:
max_len = len(max(X, key=lambda x: len(x)))

In [69]:
X_nums = np.zeros((len(X),max_len), dtype='int')
for i,x in enumerate(X):
    for j,w in enumerate(x.split()):
        idx = word2index.get(w, 1)
        X_nums[i][j] = idx

In [70]:
np.save('../out/clean-train-embedded', X_nums)

In [53]:
X_train = load_tweets('../data/clean_test.txt')

In [63]:
X_train_nums = np.zeros((len(X_train),max_len), dtype='int')
for i,x in enumerate(X_train):
    for j,w in enumerate(x.split()):
        idx = word2index.get(w, 1)
        X_train_nums[i][j] = idx

In [71]:
np.save('../out/clean-test-embedded', X_train_nums)

* * *

In [56]:
from utils import *
import pickle

In [2]:
X,y = load_embeddings('../out/embeddings_train.npy', '../out/clean_train_labels.npy')

In [3]:
y = np.array(one_minus_one_labels_to_one_zero(y))

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [6]:
from sklearn.linear_model import LogisticRegression

In [7]:
model = LogisticRegression()

In [8]:
model.fit(X_train, y_train)

/home/peter/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [24]:
preds = model.predict(X_test)

In [11]:
model.score(X_test, y_test)

0.8041691942866597

In [12]:
TEST = load_embeddings('../out/embeddings_test.npy')

In [13]:
preds = model.predict(TEST)

In [15]:
preds = one_zero_labels_to_one_minus_one(preds)

In [17]:
generate_submission('../out/sub_lr.csv', preds)

In [57]:
with open('../models/logistic_regression.pkl', 'wb') as f:
    pickle.dump(model, f)

* * *

In [59]:
max_depth = 10

In [49]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth=max_depth, random_state=42)

In [50]:
tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [51]:
y_pred = tree.predict(X_test)

In [52]:
tree.score(X_test, y_test)

0.6817404731704627

In [53]:
preds = tree.predict(TEST)
preds = one_zero_labels_to_one_minus_one(preds)

In [55]:
generate_submission('../out/sub_dt.csv', preds)

In [60]:
with open('../models/decision_tree_'+str(max_depth)+'.pkl', 'wb') as f:
    pickle.dump(tree, f)

* * *

In [64]:
from sklearn.svm import SVC
svm = SVC(random_state=42)

In [ ]:
svm.fit(X_train, y_train)

/home/peter/.local/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [ ]:
svm.score(X_test, y_test)

In [ ]:
preds = svm.predict(TEST)
preds = one_zero_labels_to_one_minus_one(preds)

In [ ]:
generate_submission('../out/sub_svm.csv', preds)

In [ ]:
with open('../models/svm.pkl', 'wb') as f:
    pickle.dump(svm, f)